In [1]:
import pymc_model as pm
import pymc
import numpy as np
import pandas as pd
import copy
import matplotlib.pyplot as plt
import latin_hypercube as lh
import xarray as xr

This study is only focusing on glaciers ['03473'] in region [15].
(1, 882) (882,)
[ 0.] (1,)


/home/tushar/anaconda3/envs/pymc/lib/python3.6/site-packages/pandas/core/indexing.py:1472: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


In [2]:
def process(df):
    df['runs'] = np.arange(len(df))
    df = df.set_index('runs')
    df = df.drop('sorted_index', 1)
    df.columns.name = 'variables'
    return df

def to_data_array(df, name):
    da = xr.DataArray(df)
    da.name = name
    return da

In [6]:
s1 = pymc.database.pickle.load('testlength1000.pickle')
s3 = pymc.database.pickle.load('testlength3000.pickle')
s5 = pymc.database.pickle.load('testlength5000.pickle')
s10 = pymc.database.pickle.load('testlength10000.pickle')
s20 = pymc.database.pickle.load('testlength20000.pickle')
s30 = pymc.database.pickle.load('testlength30000.pickle')
sample_dict = {'1000':s1, '3000':s3, '5000':s5, '10000':s10, '20000':s20, '30000':s30}

In [7]:
dict_of_df = {}

for key, value in sample_dict.items():
    tempchange = value.trace('tempchange')[:]
    precfactor = value.trace('precfactor')[:]
    ddfsnow = value.trace('ddfsnow')[:]
    massbal = value.trace('mass_bal')[:]
    result = lh.sample2(tempchange=tempchange, precfactor=precfactor,
                     ddfsnow=ddfsnow, massbal=massbal, samples=300)
    mean = np.mean(result['massbal'])
    std = np.std(result['massbal'])
    print(key + ' samples, mean:', mean, 'std:', std)
    dict_of_df[key] = result

1000 samples, mean: -0.5224877351033013 std: 0.22630761550317066
3000 samples, mean: -0.5335146821604339 std: 0.22945675933476237
5000 samples, mean: -0.540805165934963 std: 0.24589648346107287
10000 samples, mean: -0.5424262147664878 std: 0.23946520200091354
20000 samples, mean: -0.5379736835992974 std: 0.23696709955570783
30000 samples, mean: -0.5374212900428693 std: 0.23570066754099978


In [23]:
da_dict = {}

for key, value in dict_of_df.items():
    df = process(value)
    da = to_data_array(df, key)
    da_dict[key] = da

In [24]:
da_dict

{'1000': <xarray.DataArray '1000' (runs: 300, variables: 4)>
 array([[-0.161266,  0.838956,  0.005501, -0.848848],
        [-0.32146 ,  0.838956,  0.004797, -0.714395],
        [-0.715481,  0.838956,  0.001886, -0.124318],
        ..., 
        [ 0.391599,  1.336333,  0.00254 , -0.284081],
        [ 0.391599,  1.336333,  0.00605 , -0.737278],
        [ 0.568451,  1.336333,  0.00605 , -0.864915]])
 Coordinates:
   * runs       (runs) int64 0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18 ...
   * variables  (variables) object 'tempchange' 'precfactor' 'ddfsnow' 'massbal',
 '3000': <xarray.DataArray '3000' (runs: 300, variables: 4)>
 array([[  8.787521e-01,   1.858787e+00,   4.471895e-03,  -4.428125e-01],
        [  6.833473e-01,   8.303675e-01,   2.102684e-03,  -9.831374e-01],
        [  8.778726e-01,   8.303675e-01,   2.102684e-03,  -1.108236e+00],
        ..., 
        [ -1.259003e+00,   6.555924e-01,   5.472549e-03,  -4.777495e-01],
        [  4.588274e-01,   1.146194e+00,   5.472549e-

Experiment with creating a DataArray

In [25]:
ds = xr.Dataset(da_dict)

In [27]:
ds.to_netcdf('../MCMC_Data/testfile1.nc')

In [28]:
ds1 = xr.open_dataset('../MCMC_Data/testfile1.nc')

In [32]:
ds1['30000'].to_pandas()

variables,tempchange,precfactor,ddfsnow,massbal
runs,,,,
0,0.299445,1.335553,0.003112,-0.346919
1,-0.729413,0.638450,0.005428,-0.754095
2,0.746308,1.853016,0.004003,-0.269615
3,-0.073889,1.306321,0.004479,-0.321977
4,0.133630,1.039410,0.002521,-0.478865
5,0.151634,0.878990,0.002434,-0.673038
6,-0.043159,1.170329,0.006643,-0.642975
7,0.509946,0.687600,0.001643,-0.939631
8,0.239987,0.925940,0.002751,-0.725571
